In [1]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import timm
import numpy as np
import pandas as pd
import os
import faiss
from tqdm import tqdm
from collections import Counter
from torch.utils.data import Subset
from collections import defaultdict
from sklearn.metrics import silhouette_score
import csv
from pathlib import Path

from matplotlib import pyplot as plt
import random
from PIL import Image

/home/jme3qd/.conda/envs/dsvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tile_image_nxn(img, tiles_per_side=3, target_size=518):
    w, h = img.size

    tile_w = w / tiles_per_side
    tile_h = h / tiles_per_side

    tiles = []

    for row in range(tiles_per_side):
        for col in range(tiles_per_side):
            left = int(col * tile_w)
            top = int(row * tile_h)
            right = int((col + 1) * tile_w)
            bottom = int((row + 1) * tile_h)

            tile = img.crop((left, top, right, bottom))
            tile = tile.resize((target_size, target_size), Image.BICUBIC)
            tiles.append(tile)

    return tiles

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
])

In [4]:
class QuadratNxNDataset(Dataset):
    def __init__(self, root, transform, tiles_per_side=3, target_size=518):
        self.paths = sorted([str(p) for p in Path(root).glob("*.*")])
        self.transform = transform
        self.tiles_per_side = tiles_per_side
        self.target_size = target_size

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")

        tiles = tile_image_nxn(
            img,
            tiles_per_side=self.tiles_per_side,
            target_size=self.target_size
        )

        tiles = [self.transform(t) for t in tiles]
        tiles = torch.stack(tiles)  # shape [N*N, 3, 518, 518]

        return tiles, path


In [5]:
train_dataset = datasets.ImageFolder("/scratch/jme3qd/data/plantclef2025/images_max_side_800", transform=transform)

n_samples = 20000
indices = np.random.choice(len(train_dataset), n_samples, replace=False)
train_subset = Subset(train_dataset, indices)

# %%
#train_loader = DataLoader(train_subset, batch_size=32, shuffle=False, num_workers=4)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4)


/home/jme3qd/.conda/envs/dsvenv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
from pathlib import Path
quadrat_path = "/scratch/jme3qd/data/plantclef2025/data/PlantCLEF/PlantCLEF2025/DataOut/test/package/images"

quadrat_loader = DataLoader(
    QuadratNxNDataset(
        quadrat_path,
        transform,
        tiles_per_side=5,
        target_size=518
    ),
    batch_size=1,
    shuffle=False,
    num_workers=4
)

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = timm.create_model("timm/vit_base_patch14_reg4_dinov2.lvd142m", pretrained=True)
#model = timm.create_model("vit_base_patch16_224", pretrained=True)
checkpoint_path = '/home/jme3qd/Downloads/model_best.pth.tar'
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False) # Load to CPU first

if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint # Assume the checkpoint itself is the state_dict

# 3. Load the state dictionary into the model
model.load_state_dict(state_dict,strict=False)

model.eval().to(device)
for p in model.parameters():
    p.requires_grad = False

In [8]:
def extract_embeddings(dataloader):
    all_embs, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(dataloader):
            imgs = imgs.to(device)
            feats = model.forward_features(imgs)
            if feats.ndim == 3:
                cls_embs = feats[:, 0, :]  # [CLS] token
            else:
                cls_embs = feats
            all_embs.append(cls_embs.cpu().numpy())
            all_labels.append(labels.numpy())
    return np.concatenate(all_embs), np.concatenate(all_labels)

train_file = "slurm/embs_train.npz"
if os.path.exists(train_file):
    with np.load(train_file) as data:
        train_embs = data['embs']
        train_labels = data['labels']
        print("Train embeddings loaded from", train_file)
else:
    train_embs, train_labels = extract_embeddings(train_loader)
    np.savez(train_file, embs=train_embs, labels=train_labels)


Train embeddings loaded from slurm/embs_train.npz


In [14]:
def extract_unlabeled_embeddings(dataloader, model, device):
    all_embs, all_paths = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader):

            imgs = batch[0]      # shape (B, T, 3, 518, 518)
            paths = batch[1]     # list of length B

            # Flatten tiles:
            # imgs: (B, T, C, H, W) → (B*T, C, H, W)
            B, T, C, H, W = imgs.shape
            imgs = imgs.view(B * T, C, H, W).to(device)

            feats = model.forward_features(imgs)
            cls_embs = feats[:, 0, :] if feats.ndim == 3 else feats

            # Save embeddings
            all_embs.append(cls_embs.cpu().numpy())

            # Save one path for every tile
            # If image X has 25 tiles → repeat the path 25 times
            for p in paths:
                all_paths.extend([p] * T)

    return np.concatenate(all_embs), all_paths


filename = "slurm/quadrat_embs_5x5.npz"
if os.path.exists(filename):
    with np.load(filename) as data:
        quadrat_embs = data['embs']
        quadrat_paths = data['paths']
        print("Quadrat embeddings loaded from", filename)
else:
    quadrat_embs, quadrat_paths = extract_unlabeled_embeddings(quadrat_loader, model, device)
    np.savez(filename, embs=quadrat_embs, paths=quadrat_paths)

100%|██████████| 2105/2105 [59:16<00:00,  1.69s/it] 


In [13]:
batch = next(iter(quadrat_loader))
imgs = batch[0]
meta = batch[1]

print("imgs shape:", imgs.shape)
print("first meta item:", meta[0])
print("type of meta item:", type(meta[0]))

imgs shape: torch.Size([1, 25, 3, 518, 518])
first meta item: /scratch/jme3qd/data/plantclef2025/data/PlantCLEF/PlantCLEF2025/DataOut/test/package/images/2024-CEV3-20240602.jpg
type of meta item: <class 'str'>


In [ ]:
faiss.normalize_L2(quadrat_embs)
faiss.normalize_L2(train_embs)
index = faiss.IndexFlatIP(train_embs.shape[1])
index.add(train_embs)
D, I = index.search(quadrat_embs, k=5)

In [ ]:
quadrat_tile_predictions = defaultdict(list)

with torch.no_grad():
    for tiles, path in tqdm(quadrat_loader):
        path = path[0]
        tiles = tiles.to(device)

        T = tiles.shape[1]
        tiles = tiles.view(T, 3, 518, 518)

        feats = model.forward_features(tiles)
        cls = feats[:, 0, :].cpu().numpy()

        k = 5
        distances, indices = index.search(cls, k)

        tile_labels = train_labels[indices]
        quadrat_tile_predictions[path] = tile_labels


  0%|          | 0/2105 [00:00<?, ?it/s]/home/jme3qd/.conda/envs/dsvenv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  6%|▌         | 126/2105 [08:08<2:07:48,  3.87s/it]


KeyboardInterrupt: 

In [ ]:
quadrat_final_predictions = {}

for path, tile_knn_labels in quadrat_tile_predictions.items():
    flat = tile_knn_labels.flatten()
    counts = Counter(flat)

    total_votes = len(flat)
    threshold = 0.05 * total_votes

    preds = [s for s, c in counts.items() if c >= threshold]
    quadrat_final_predictions[path] = preds

In [ ]:
with open("submission.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["quadrat_id", "species_ids"])

    for path, species_list in quadrat_final_predictions.items():
        quadrat_id = Path(path).stem  # filename without extension

        # Convert list → "[1, 2, 3]"
        species_str = "[" + ", ".join(str(s) for s in species_list) + "]"

        writer.writerow([quadrat_id, species_str])


In [ ]:

sil_score = silhouette_score(train_embs, train_labels, metric='cosine')
print(f"Silhouette score: {sil_score:.3f}")


KeyboardInterrupt: 

In [ ]:


D_train, I_train = index.search(train_embs, k=5)
purity = np.mean([
    np.mean(train_labels[i] == train_labels[nbrs]) 
    for i, nbrs in enumerate(I_train)
])
print(f"Neighborhood purity (k=5): {purity:.3f}")


In [ ]:
k = 5
idx = random.randint(0, len(quadrat_paths)-1)
nbrs = I[idx]
fig, axes = plt.subplots(1, k+1, figsize=(15, 4))
axes[0].imshow(Image.open(quadrat_paths[idx]))
axes[0].set_title("Quadrat tile")
for j, n in enumerate(nbrs):
    axes[j+1].imshow(Image.open(train_dataset.samples[n][0]))
    axes[j+1].set_title(train_dataset.classes[train_labels[n]])
plt.savefig(f"quadrat_{idx}.png")